#### **整图分类流程**：

step 1: 准备一个批次的图；

step 2: 在这个批次的图上进行消息传递以更新节点或边的特征；

step 3: 将一张图里的节点或边特征聚合成整张图的图表示；

step 4: 根据任务设计分类层。

In [1]:
import dgl
import torch

g1 = dgl.graph(([0, 1], [1, 0]))
g1.ndata['h'] = torch.tensor([1., 2.])
g2 = dgl.graph(([0, 1], [1, 2]))
g2.ndata['h'] = torch.tensor([1., 2., 3.])

dgl.readout_nodes(g1, 'h')

bg = dgl.batch([g1, g2])
dgl.readout_nodes(bg, 'h')

C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\18438\AppData\Local\Continuum\anaconda3\envs\dgl\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Using backend: pytorch


tensor([3., 6.])

最后，批次化图中的每个节点或边特征张量均通过将所有图上的相应特征拼接得到。

In [2]:
bg.ndata['h']

tensor([1., 2., 1., 2., 3.])

模型定义

In [3]:
import dgl.nn.pytorch as dglnn
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.conv1 = dglnn.GraphConv(in_dim, hidden_dim)
        self.conv2 = dglnn.GraphConv(hidden_dim, hidden_dim)
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h):
        # 应用图卷积和激活函数
        h = F.relu(self.conv1(g, h))
        h = F.relu(self.conv2(g, h))
        with g.local_scope():
            g.ndata['h'] = h
            # 使用平均读出计算图表示
            hg = dgl.mean_nodes(g, 'h')
            return self.classify(hg)

数据加载, 使用自带的整图分类数据集。

In [8]:
import dgl.data
dataset = dgl.data.GINDataset('MUTAG', False)

整图分类数据集里的每个数据点是一个图和它对应标签的元组。  
为提升数据加载速度， 用户可以调用GraphDataLoader，从而以小批次遍历整个图数据集。

In [ ]:
from dgl.dataloading import GraphDataLoader
dataloader = GraphDataLoader(
    dataset,
    batch_size=1024,
    drop_last=False,
    shuffle=True)

训练过程包括遍历dataloader和更新模型参数的部分

In [ ]:
import torch.nn.functional as F

# 这仅是个例子，特征尺寸是7
model = Classifier(7, 20, 5)
opt = torch.optim.Adam(model.parameters())
for epoch in range(20):
    for batched_graph, labels in dataloader:
        feats = batched_graph.ndata['attr']
        logits = model(batched_graph, feats)
        loss = F.cross_entropy(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()